In [1]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0,
)

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are an expert product reviewer."),
    ("human", "List the main features of the product {product_name}."),
  ]
)

In [5]:
def analyze_pros(features):
  pros_template = ChatPromptTemplate.from_messages(
    [
      ("system", "You are an expert product reviewer."),
      ("human", "Given these features: {features}, list the pros of these features."),
    ]
  )
  return pros_template.format_prompt(features=features)

In [6]:
def analyze_cons(features):
  cons_template = ChatPromptTemplate.from_messages(
    [
      ("system", "You are an expert product reviewer."),
      ("human", "Given these features: {features}, list the cons of these features."),
    ]
  )
  return cons_template.format_prompt(features=features)

In [7]:
def combine_pros_cons(pros, cons):
  return f"Pros:\n{pros}\n\nCons:\n{cons}"

In [10]:
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

pros_branch_chain = (
  RunnableLambda(lambda x: analyze_pros(x)) | llm | StrOutputParser()
)

In [11]:
cons_branch_chain = (
  RunnableLambda(lambda x: analyze_cons(x)) | llm | StrOutputParser()
)

In [12]:
from langchain_core.runnables import RunnableParallel
chain = (
  prompt_template
  | llm
  | StrOutputParser()
  | RunnableParallel(branches={"pros": pros_branch_chain, "cons": cons_branch_chain})
  | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

In [13]:
print(chain.invoke({"product_name": "MacBook Pro"}))

Pros:
The MacBook Pro's features offer numerous advantages, particularly for professionals and power users. Here are the pros associated with each feature:

1. **Display**:
   - **Retina Display with True Tone**: Provides stunning visuals with accurate color representation, making it ideal for graphic design, photo editing, and video production.
   - **High Resolution**: The high pixel density ensures sharp images and text, enhancing the overall viewing experience.
   - **P3 Wide Color Gamut**: Delivers vibrant and true-to-life colors, which is essential for creative work.
   - **ProMotion Technology**: The 120Hz refresh rate allows for smoother scrolling and responsiveness, improving user experience during tasks like gaming or video editing.

2. **Processor**:
   - **Apple Silicon**: Offers exceptional performance and efficiency, allowing for faster processing and better battery life compared to traditional Intel processors.
   - **Multi-core CPUs and GPUs**: Provides the power needed